In [131]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw

mesh = ngs.Mesh('team10_2d.vol')
mesh.Curve(2)
# mesh.Refine()

# print("using 2D mesh with ne=", mesh.ne, " elements and nv=", mesh.nv, " points")


In [132]:
from ngsolve.krylovspace import CGSolver

deg = 2

# HCurl = ngs.HCurl(mesh, order = deg, dirichlet = "ambient_face")
HCurl = ngs.HCurl(mesh, order = deg)
H1 = ngs.H1(mesh, order = deg + 1)

X = HCurl*H1

(u,p),(v,q) = X.TnT()

eps = 1e-8

K = ngs.BilinearForm(X, condense = True)
K += (ngs.curl(u)*ngs.curl(v) - v*ngs.grad(p) +ngs.grad(q)*u)*ngs.dx
C = ngs.Preconditioner(K, type = "local")

total_current = 1000/0.0025
cfJ = mesh.MaterialCF({"coil_plus": total_current, "coil_minus": -total_current}, default = 0)


f = ngs.LinearForm(X)
f += cfJ*ngs.curl(v)*ngs.dx()

with ngs.TaskManager():
    K.Assemble()
    f.Assemble()

HsA = ngs.GridFunction(X)

invS = K.mat.Inverse(freedofs = X.FreeDofs(coupling=True))
ext = ngs.IdentityMatrix() + K.harmonic_extension
extT = ngs.IdentityMatrix() + K.harmonic_extension_trans
invA =  ext @ invS @ extT + K.inner_solve

HsA.vec.data = invA*f.vec
# HsA.vec.data = K.mat.Inverse(X.FreeDofs())*f.vec

Hs = HsA.components[0]

# # with ngs.TaskManager():
# #     iterativeSolver = CGSolver(K.mat, pre = C.mat, tol  = 1e-8,  maxiter = 1000)
# #     J.vec.data = iterativeSolver * f.vec

# J.vec.FV()

import numpy as np
actual_ndof = np.argwhere(X.FreeDofs(True)).size
print("Using 2D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and ", mesh.nedge, "edges and " ,X.ndof, "DOFs. Actual DOFs after condensation:", actual_ndof, ".\n ")

Using 2D mesh with ne= 33576 elements and nv= 16987 points and  50562 edges and  404101 DOFs. Actual DOFs after condensation: 269797 .
 


In [133]:
# Draw(Hs, mesh)
# Draw(ngs.curl(Hs), mesh)

In [134]:
# from ngsolve.krylovspace import CGSolver

# deg = 3

# # HCurl = ngs.HCurl(mesh, order = deg, dirichlet = "ambient_face")
# HCurl = ngs.HCurl(mesh, order = deg, nograds = True)

# X = HCurl

# u, v = X.TnT()

# eps = 0 #1e-5

# K = ngs.BilinearForm(X)
# K += (ngs.curl(u)*ngs.curl(v)+eps*u*v)*ngs.dx
# C = ngs.Preconditioner(K, type = "local")

# total_current = 1000/0.0025
# cfJ = mesh.MaterialCF({"coil_plus": total_current, "coil_minus": -total_current}, default = 0)


# f = ngs.LinearForm(X)
# f += cfJ*ngs.curl(v)*ngs.dx()

# with ngs.TaskManager():
#     K.Assemble()
#     f.Assemble()

# Hs = ngs.GridFunction(X)

# # invS = K.mat.Inverse(freedofs = X.FreeDofs(coupling=True))
# # ext = ngs.IdentityMatrix() + K.harmonic_extension
# # extT = ngs.IdentityMatrix() + K.harmonic_extension_trans
# # invA =  ext @ invS @ extT + K.inner_solve

# # HsA.vec.data = invA*f.vec
# # Hs.vec.data = K.mat.Inverse(X.FreeDofs())*f.vec

# with ngs.TaskManager():
#     iterativeSolver = CGSolver(K.mat, pre = C.mat, tol  = 1e-8,  maxiter = 100000)
#     Hs.vec.data = iterativeSolver * f.vec

# # J.vec.FV()

# import numpy as np
# actual_ndof = np.argwhere(X.FreeDofs()).size
# print("Using 2D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and ", mesh.nedge, "edges and " ,X.ndof, "DOFs. Actual DOFs after condensation:", actual_ndof, ".\n ")

In [135]:
# Draw(Hs, mesh)
# Draw(ngs.curl(Hs), mesh)